<a href="https://colab.research.google.com/github/sthalles/logistic-regression/blob/master/Binary_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
from numpy.linalg import pinv, inv
import matplotlib.pyplot as plt

In [0]:
class DataSet:
  def __init__(self, data, targets, valid_classes=None):
    if valid_classes is None:
      self.valid_classes = np.unique(targets)
    else:
      self.valid_classes = valid_classes
    #print(self.valid_classes)
    self.number_of_classes = len(self.valid_classes)
    self.data = self.to_dict(data, targets)
    
    total = 0
    for i in self.data.keys():
      print("Class {0} # of records: {1}".format(i,len(self.data[i])))
      total += len(self.data[i])
    print("Total:",total)
    
  def to_dict(self, data, targets):
    data_dict = {}
    for x, y in zip(data, targets):
      if y in self.valid_classes:
        if y not in data_dict:
          data_dict[y] = [x.flatten()]
        else:
          data_dict[y].append(x.flatten())

    for i in self.valid_classes:
      data_dict[i] = np.asarray(data_dict[i])

    return data_dict

  def get_data_by_class(self, class_id):
    if class_id in self.valid_classes:
      return self.data[class_id]
    else:
      raise ("Class not found.")

  def get_data_as_dict(self):
    return self.data

  def get_all_data(self):
    data = []
    labels = []
    for label, class_i_data in self.data.items():
      data.extend(class_i_data)
      labels.extend(class_i_data.shape[0] * [label])
    data = np.asarray(data)
    labels = np.asarray(labels)
    return data, np.expand_dims(labels,1)

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [0]:
train_dataset = DataSet(x_train, y_train, valid_classes=[0,1])
inputs, targets = train_dataset.get_all_data()

test_dataset = DataSet(x_test, y_test, valid_classes=[0,1])
test_inputs, test_targets = test_dataset.get_all_data()

Class 0 # of records: 5923
Class 1 # of records: 6742
Total: 12665
Class 1 # of records: 1135
Class 0 # of records: 980
Total: 2115


In [0]:
print("inputs shape:",inputs.shape)
print("targets shape:",targets.shape)

inputs shape: (12665, 784)
targets shape: (12665, 1)


- For an M-dimensional feature space φ, this model has M adjustable parameters


In [0]:
def generate_R(predictions):
  # N × N diagonal matrix R with elements Rnn = yn(1−yn)
  R = np.zeros((predictions.shape[0],predictions.shape[0]))
  for i in range(R.shape[0]):
    #print(predictions[i])
    R[i][i] = predictions[i] * (1-predictions[i])
  return R

def hessian(x,R):
  return np.dot(x.T,np.dot(R,x))
    
def grad(y,t,x):
  # y: models predictions (Batch,)
  # t: dataset targets (Batch,)
  return np.dot(x.T, (y - t))
  
  
class LogisticRegression:
  def __init__(self,fit_intercept=True):
    self.fit_intercept = fit_intercept
    self.W = None
    
  
  def score(self,x,y):
    
    if self.fit_intercept:
      x = self.add_intercept(x)    
      
    logits = self.forward(x)
    predictions = [1 if t >= 0.5 else 0 for t in logits]
#     print("predictions:",predictions)
#     print("targets:",y)
    return np.sum(np.squeeze(predictions) == np.squeeze(y)) / len(y)

  def sigmoid(self,x):
    # Under rather general assumptions, the posterior probability of class C1 can be written 
    # as a logistic sigmoid acting on a linear function of the feature vector φ so that
    return 1. / (1. + np.exp(-x))
    
  def forward(self,x):
    
    if self.W is None:
      self.W = np.full((x.shape[1],1), 0.01) #np.random.rand(x.shape[1],1)
      print("W.shape",self.W.shape)
    linear = np.dot(x,self.W)
    
    print("Weights: Min: {0} Max: {1}".format(np.min(self.W), np.max(self.W)))
    print("Probabilities: Min: {0} Max: {1}".format(np.min(linear), np.max(linear)))
    return self.sigmoid(linear) # the order dot(x,W) seems correct

  def add_intercept(self,x):
    # generate a NxM design matrix, with an added column of 1
    const = np.ones((x.shape[0],1))
    return np.concatenate((const,x),axis=1)

  def fit(self,x,y,iterations=2):
    
    if self.fit_intercept:
      x = self.add_intercept(x)
      
    for i in range(iterations):
      predictions = self.forward(x)
      R = generate_R(predictions)
      H = hessian(x,R)
      print("Hessian:",H.shape)
      gradients = grad(predictions,y,x)
      invH = pinv(H)
      #W_new = self.W - 0.2 * gradients
      self.W = self.W - np.dot(invH, gradients)

In [0]:
clf = LogisticRegression(fit_intercept=True)

In [0]:
BATCH_SIZE=inputs.shape[0]
inputs = inputs[:BATCH_SIZE,]
targets = targets[:BATCH_SIZE,]
print(inputs.shape, targets.shape)
logits = clf.forward(inputs)
print("logits.shape:",logits.shape)
  

(12665, 784) (12665, 1)
W.shape (784, 1)
Weights: Min: 0.01 Max: 0.01
Probabilities: Min: 0.19945098039215686 Max: 3.1169803921568593
logits.shape: (12665, 1)


- Uses the Newton-Raphson iterative optimization scheme to optimize the values of W

In [0]:
R = generate_R(logits)
print(R.shape)

(12665, 12665)


In [0]:
# H1 = hessian(inputs,R)
# print("H1.shape:",H1.shape)
# print(np.nonzero(H1))

# H2 = hessian_2(batch_x,R)
# print(np.nonzero(H2))
# print(H1.shape,H2.shape)

# print("Following the associativity property, H1 and H2 should be equal")
# print(np.all(H1 == H2)) # True (problelm with precision though)

In [0]:
gradients = grad(logits,targets,inputs)
print("grads.shape:",gradients.shape)

grads.shape: (784, 1)


In [0]:
# Note that the predictions, are always between 0 and 1 - due to the sigmoid function squashing
print(np.min(logits),np.max(logits))

0.5496981021461362 0.957587760803051


In [0]:
clf = LogisticRegression(fit_intercept=True)
clf.fit(inputs,targets)

W.shape (785, 1)
Weights: Min: 0.01 Max: 0.01
Probabilities: Min: 0.20945098039215687 Max: 3.1269803921568595
Hessian: (785, 785)
Weights: Min: -131.2646916891356 Max: 73.5545290216585
Probabilities: Min: -9.67115827495726 Max: 3.361295299510244
Hessian: (785, 785)


In [0]:
acc = clf.score(inputs,targets)
print(acc)

Weights: Min: -95.13851871653901 Max: 53.394425209318584
Probabilities: Min: -12.680541874001982 Max: 5.392235406124617
0.9996052112120016


In [0]:
acc = clf.score(test_inputs,test_targets)
print(acc)

Weights: Min: -95.13851871653901 Max: 53.394425209318584
Probabilities: Min: -10.969357161091907 Max: 38.2660259135658
0.9985815602836879
